In [188]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score, f1_score, precision_recall_curve
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer, load_wine
from ucimlrepo import fetch_ucirepo
import torch
import sys
import openml
from idistill.whitebox_figs import FIGSRegressorCV, FIGSRegressor


In [166]:
d = fetch_ucirepo(id=697)
X = d.data.features
y = d.data.targets

In [167]:
y = y.Target.map({'Dropout':0,'Graduate':1, 'Enrolled':2})
y

0       0
1       1
2       0
3       1
4       1
       ..
4419    1
4420    0
4421    0
4422    1
4423    1
Name: Target, Length: 4424, dtype: int64

In [168]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train.shape, X_test.shape

((2964, 36), (1460, 36))

In [169]:
# Initialize a classifier
clf = TabPFNClassifier(ignore_pretraining_limits=True)
clf.fit(X_train, y_train)

# # Predict probabilities
# prediction_probabilities = clf.predict_proba(X_test)
# print("ROC AUC:", roc_auc_score(y_test, prediction_probabilities[:, 1]))

# Predict labels
predictions = clf.predict(X_test)
print("Accuracy", accuracy_score(y_test, predictions))

Accuracy 0.7705479452054794


In [170]:
y_train_preds = clf.predict_proba(X_train)

In [184]:
accuracy_score(np.argmax(y_train_preds, axis = 1), y_train)

0.8451417004048583

In [172]:
figs = FIGSRegressor(max_rules = 100, max_trees = 20, max_depth = 3, min_impurity_decrease=0.0001)
x = xgb.XGBRegressor(n_estimators = 20, max_depth=3)
figs.fit(X_train, y_train_preds)
x.fit(X_train, y_train_preds)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=20, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [187]:
figs.complexity_, len(figs.trees_)

(64, 20)

In [174]:
accuracy_score(np.argmax(y_train_preds, axis = 1), np.argmax(figs.predict(X_train), axis = 1))

0.9143049932523617

In [185]:
accuracy_score(y_train, np.argmax(figs.predict(X_train), axis = 1))

0.786774628879892

In [176]:
accuracy_score(y_test, np.argmax(figs.predict(X_test), axis = 1))

0.7486301369863013

In [177]:
accuracy_score(predictions, np.argmax(figs.predict(X_test), axis = 1))

0.9164383561643835

In [178]:
#xgb
accuracy_score(np.argmax(y_train_preds, axis = 1), np.argmax(x.predict(X_train), axis = 1))

0.9166666666666666

In [186]:
accuracy_score(y_train, np.argmax(x.predict(X_train), axis = 1))

0.7901484480431848

In [179]:
accuracy_score(y_test, np.argmax(x.predict(X_test), axis = 1))

0.7568493150684932

In [180]:
accuracy_score(predictions, np.argmax(x.predict(X_test), axis = 1))

0.9191780821917809

In [181]:
num_rules = 0
trees = x.get_booster().get_dump(with_stats=True)

for tree in trees:
    num_rules += len([line for line in tree.splitlines() if 'leaf=' not in line])

print(num_rules, figs.complexity_/num_rules)

417 0.15347721822541965


In [182]:
figs.complexity_, len(figs.trees_)

(64, 20)

In [129]:
clf.predict_proba(X_train)

array([[4.44500292e-05, 9.99951601e-01, 3.99124974e-06],
       [9.10958217e-04, 9.99036431e-01, 5.26039912e-05],
       [9.99973834e-01, 2.09885402e-05, 5.16762066e-06],
       [3.95135976e-05, 9.99941111e-01, 1.93853775e-05],
       [3.46737988e-06, 9.99989390e-01, 7.12690507e-06],
       [4.51598316e-04, 9.99541640e-01, 6.77389289e-06],
       [3.55625139e-06, 6.17750993e-06, 9.99990284e-01],
       [3.01912928e-06, 4.18654410e-04, 9.99578357e-01],
       [9.99967992e-01, 3.08880408e-05, 1.11585007e-06],
       [1.41482424e-05, 9.99972701e-01, 1.31836887e-05],
       [3.74329784e-06, 1.82660253e-04, 9.99813616e-01],
       [3.76009302e-06, 1.46144575e-05, 9.99981642e-01],
       [2.72068253e-04, 9.73517776e-01, 2.62101833e-02],
       [1.81058794e-03, 9.98148441e-01, 4.09310815e-05],
       [9.99801397e-01, 1.95370842e-04, 3.16726209e-06],
       [1.55526377e-05, 9.99982238e-01, 2.22830408e-06],
       [6.80231051e-06, 6.83423481e-04, 9.99309719e-01],
       [2.62354934e-06, 5.78151